In [1]:
!pip install transformers

In [9]:
!pip install tf_keras

In [10]:
!pip install tensorflow

  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 1.52.2 requires typing-extensions<5,>=4.11, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.9.2 requires typing-extensions>=4.6.1; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.23.4 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
selenium 4.25.0 requires typing_extensions~=4.9, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [11]:
from transformers import pipeline

clf = pipeline('sentiment-analysis')
result = clf('what a beautiful day!')[0]
print('감성분석 결과: %s, 감성스코어: %0.4f' % (result['label'], result['score']))

ModuleNotFoundError: No module named 'transformers'

In [6]:
from transformers import pipeline

text_generator = pipeline('text-generation')
result = text_generator('Alice was beginning to get very tired of sitting by her sister on the bank,')
print(result[0]['generated_text'])

ModuleNotFoundError: No module named 'transformers'

In [7]:
!pip install torch torchvision torchaudio

  Using cached torch-2.4.1-cp38-cp38-win_amd64.whl.metadata (27 kB)
  Using cached torchvision-0.19.1-cp38-cp38-win_amd64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.4.1-cp38-cp38-win_amd64.whl.metadata (6.5 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
Using cached torch-2.4.1-cp38-cp38-win_amd64.whl (199.4 MB)
Using cached torchvision-0.19.1-cp38-cp38-win_amd64.whl (1.3 MB)
Using cached torchaudio-2.4.1-cp38-cp38-win_amd64.whl (2.4 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-cased-finetuned-mrpc'
)

input_sentence = 'She angered me with her inappropriate commetns, rumor-spreading, and disrespectfulness at the formal dinner table'
target_sequence = 'She made me angry when she was rude at dinner'

tokens = tokenizer(input_sentence, target_sequence, return_tensors='pt')

logits = model(**tokens).logits

results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f'{label}: {int(round(results[i]*100))}%')

no: 65%
yes: 35%


In [6]:
target_sequence = 'The boy quickly ran across the finish line, seizing yet another victory'
tokens = tokenizer(input_sentence, target_sequence, return_tensors='pt')
logits = model(**tokens).logits
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f'{label}: {int(round(results[i]*100))}%')

no: 95%
yes: 5%


In [8]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

nltk.download('movie_reviews')

fileids = movie_reviews.fileids()

reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count:  1600
Test set count:  400


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained(
    'distilbert-base-uncased-finetuned-sst-2-english'
)
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased-finetuned-sst-2-english'
)

model = model.to(device)
batch_size = 10
y_pred = []

num_batch = len(y_test)//batch_size

for i in range(num_batch):
    inputs = tokenizer(
        X_test[i*batch_size:(i+1)*batch_size],
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = inputs.to(device)
    logits = model(**inputs).logits
    pred = F.softmax(logits, dim=1)
    results = pred.cpu().detach().numpy().argmax(axis=1)
    y_pred.extend(results.tolist())

torch.cuda.empty_cache()



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

NLTK 영화리뷰 감성분석 정확도: 184


In [13]:
score = sum(y_test == np.array(y_pred))/len(y_test)
print('NLTK 영화리뷰 감성분석 정확도:',score)

NLTK 영화리뷰 감성분석 정확도: 0.8425
